In [2]:
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import numpy.ma as ma
import pyart
import time
import math
import singledop
from copy import deepcopy

from csu_radartools import (csu_misc, csu_kdp)

import gzip
%matplotlib inline

print pyart.__version__

1.7.0.dev+f4b2779


In [14]:
engine = sqlalchemy.create_engine('postgresql://alexandertam@localhost/postgres')
weather = pd.read_sql('SELECT * FROM weather',con = engine)
#weather = weather[weather["IsTornado"] == 1]
weather.drop_duplicates(["Filename","TornadoTime"])
weather.reset_index()
del weather["index"]
weather

,Filename,IsTornado,Episode_ID,Event_ID,TornadoTime,VolumeTime,OriginalTime
0,KHTX20150630_154127_V06.gz,0,97889,588549,15:45:00,15:41:27,10:45:00
1,KHTX20150630_134236_V06.gz,0,97889,588550,13:45:00,13:42:36,08:45:00
2,KHTX20150626_184539_V06.gz,0,97885,588529,18:50:00,18:45:39,13:50:00
3,KHTX20150626_194537_V06.gz,0,97885,588533,19:48:00,19:45:37,14:48:00
4,KBMX20150125_232601_V06.gz,0,92860,556884,23:30:00,23:26:01,17:30:00
5,KBMX20150125_233152_V06.gz,0,92860,556885,23:37:00,23:31:52,17:37:00
6,KBMX20150126_001544_V06.gz,0,92860,556886,00:17:00,00:15:44,18:17:00
7,KBMX20150126_003600_V06.gz,0,92860,556887,00:38:00,00:36:00,18:38:00
8,KBMX20150126_002533_V06.gz,0,92860,556888,00:30:00,00:25:33,18:30:00
9,KBMX20150126_003600_V06.gz,0,92860,556889,00:38:00,00:36:00,18:38:00


In [15]:
def extract_unmasked_data(radar, field, bad=-32768):
    """Simplify getting unmasked radar fields from Py-ART"""
    return radar.fields[field]['data'].filled(fill_value=bad)

In [16]:
def add_field_to_radar_object(field, radar, field_name='FH', units='unitless', 
                              long_name='Hydrometeor ID', standard_name='Hydrometeor ID',
                              dz_field='ZC'):
    """
    Adds a newly created field to the Py-ART radar object. If reflectivity is a masked array,
    make the new field masked the same as reflectivity.
    """
    fill_value = -32768
    masked_field = np.ma.asanyarray(field)
    masked_field.mask = masked_field == fill_value
    if hasattr(radar.fields[dz_field]['data'], 'mask'):
        setattr(masked_field, 'mask', 
                np.logical_or(masked_field.mask, radar.fields[dz_field]['data'].mask))
        fill_value = radar.fields[dz_field]['_FillValue']
    field_dict = {'data': masked_field,
                  'units': units,
                  'long_name': long_name,
                  'standard_name': standard_name,
                  '_FillValue': fill_value}
    radar.add_field(field_name, field_dict, replace_existing=True)
    return radar

In [17]:
def two_panel_plot(radar, sweep=0, var1='reflectivity', vmin1=0, vmax1=65,
                   cmap1='pyart_NWSRef', units1='dBZ', gatefilter1 = None, var2='differential_reflectivity',
                   vmin2=-5, vmax2=5, cmap2='pyart_NWSRef', units2='dB', gatefilter2 = None, return_flag=False,
                   xlim=[-150,150], ylim=[-150,150]):
    display = pyart.graph.RadarDisplay(radar)
    fig = plt.figure(figsize=(13,5))
    ax1 = fig.add_subplot(121)
    display.plot_ppi(var1, sweep=sweep, vmin=vmin1, vmax=vmax1, cmap=cmap1, 
                     colorbar_label=units1, mask_outside=True, gatefilter=gatefilter1)
    display.set_limits(xlim=xlim, ylim=ylim)
    ax2 = fig.add_subplot(122)
    display.plot_ppi(var2, sweep=sweep, vmin=vmin2, vmax=vmax2, cmap=cmap2, 
                     colorbar_label=units2, mask_outside=True, gatefilter=gatefilter2)
    display.set_limits(xlim=xlim, ylim=ylim)
    
    if return_flag:
        return fig, ax1, ax2, display

In [18]:
def cleaned_radar_image(filename, radar, sweep=0, var1='reflectivity', vmin1=0, vmax1=65,
                   cmap1='pyart_NWSRef', units1='dBZ', gatefilter1 = None,  return_flag=False,
                   xlim=[-150,150], ylim=[-150,150]):
    display = pyart.graph.RadarDisplay(radar)
    fig = plt.figure(figsize=(13,5))
    ax1 = fig.add_subplot(121)
    display.plot_ppi(var1, sweep=sweep, vmin=vmin1, vmax=vmax1, cmap=cmap1, 
                     colorbar_label=units1, mask_outside=True, gatefilter=gatefilter1,colorbar_flag=True,title_flag=True)
    display.set_limits(xlim=xlim, ylim=ylim)
    display.plot_colorbar = None
    
    #ax1.axis('off')
    
    filename = filename.split('.')[0] + "_"+var1+"_sweep_" + str(sweep)
    print filename
    
    plt.savefig('../images/'+filename,bbox_inches='tight')
    plt.show()
    #plt.close(fig)
    
    if return_flag:
        return fig, ax1, ax2, display

In [19]:
def cleaned_radar_map_image(filename, radar, sweep=0, var='reflectivity', vmin=0, vmax=65,
                    cmap='pyart_NWSRef', units='dBZ', minLat = None, minLong = None, maxLat = None, maxLong = None,
                    tornadoLat = None, tornadoLong = None):
    display = pyart.graph.RadarMapDisplay(radar)

    display.plot_ppi_map(var, sweep, vmin=vmin, vmax=vmax,
                         min_lon=minLong, max_lon=maxLong, min_lat=minLat, max_lat=maxLat,
                         lon_lines=np.arange(-158, -154, .2), projection='lcc',
                         lat_lines=np.arange(69, 72, .1), resolution='h',
                         lat_0=radar.latitude['data'][0],
                         lon_0=radar.longitude['data'][0])

    # plot range rings at 10, 20, 30 and 40km
    display.plot_range_ring(10., line_style='k-')
    display.plot_range_ring(20., line_style='k--')
    display.plot_range_ring(30., line_style='k-')
    display.plot_range_ring(40., line_style='k--')

#     # plots cross hairs
#     display.plot_line_xy(np.array([-40000.0, 40000.0]), np.array([0.0, 0.0]),
#                          line_style='k-')
#     display.plot_line_xy(np.array([0.0, 0.0]), np.array([-20000.0, 200000.0]),
#                          line_style='k-')

    # Indicate the radar location with a point
    try:
        display.plot_point(tornadoLong, tornadoLat)
    except:
        print "Could not plot display tornado point."
        
    filename = filename.split('.')[0] + "_"+var+"_sweep_" + str(sweep)
    print filename
    
    plt.savefig('../images/'+filename,bbox_inches='tight')

    plt.show()

In [20]:
class BoundingBox(object):
    def __init__(self, *args, **kwargs):
        self.min_lat = None
        self.min_lon = None
        self.max_lat = None
        self.max_lon = None

In [21]:
def get_bounding_box(latitude_in_degrees, longitude_in_degrees, half_side_in_miles):
    assert half_side_in_miles > 0
    assert latitude_in_degrees >= -90.0 and latitude_in_degrees  <= 90.0
    assert longitude_in_degrees >= -180.0 and longitude_in_degrees <= 180.0

    half_side_in_km = half_side_in_miles * 1.609344
    lat = math.radians(latitude_in_degrees)
    lon = math.radians(longitude_in_degrees)

    radius  = 6371
    # Radius of the parallel at given latitude
    parallel_radius = radius*math.cos(lat)

    lat_min = lat - half_side_in_km/radius
    lat_max = lat + half_side_in_km/radius
    lon_min = lon - half_side_in_km/parallel_radius
    lon_max = lon + half_side_in_km/parallel_radius
    rad2deg = math.degrees

    box = BoundingBox()
    box.min_lat = rad2deg(lat_min)
    box.min_lon = rad2deg(lon_min)
    box.max_lat = rad2deg(lat_max)
    box.max_lon = rad2deg(lon_max)

    return (box)

In [22]:
weather["vel_max"] = np.NaN
weather["vel_min"] = np.NaN
weather["ref_max"] = np.NaN
weather["ref_avg"] = np.NaN
weather["vel_avg"] = np.NaN
weather["vel_99"] = np.NaN
weather["vel_98"] = np.NaN
weather["rel_99"] = np.NaN
weather["rel_98"] = np.NaN

In [23]:
data = weather.copy(deep=True)
# 7 is vel_max
# 8 is vel_min
# 9 is ref_max
# 10 is rev_avg
# 11 is vel_avg
# 12 is vel_99
# 13 is vel_98
# 14 is ref_99
# 15 is ref_98

In [26]:
lenOfData = len(data)-1
for i in range(lenOfData):
#for i in range(392,lenOfWeather):

    print "Percent Complete: ", round((i+1)/float(lenOfData)*100,2)
    print "Reading "+ data.iloc[i,0] + " ..."
    try:
        radar = pyart.io.read_nexrad_archive("../assets/"+ data.iloc[i,0])
    except:
        pass
    print "Cleaning ..."
    start = time.clock()
    
#     dzN = radar.get_field(0,"reflectivity")
#     drN = radar.get_field(0,"differential_reflectivity")
#     dpN = radar.get_field(0,"differential_phase")
#     dvN = radar.get_field(1,"velocity")
    
    
    dzN = extract_unmasked_data(radar, 'reflectivity')
    drN = extract_unmasked_data(radar, 'differential_reflectivity')
    dpN = extract_unmasked_data(radar, 'differential_phase')
    #dvN = extract_unmasked_data(radar, 'velocity')
    
    rng2d, az2d = np.meshgrid(radar.range['data'], radar.azimuth['data'])
    
    kdN, fdN, sdN = csu_kdp.calc_kdp_bringi(
    dp=dpN, dz=dzN, rng=rng2d/1000.0, thsd=12, gs=250.0, window=5)
    
    #generate insect mask from reflectivity and differential reflectivity
    insect_mask = csu_misc.insect_filter(dzN, drN)
    sdp_mask = csu_misc.differential_phase_filter(sdN, thresh_sdp=13)
    
    #apply mask to respective fields
    bad = -32768
    dz_insect = 1.0 * dzN
    dz_insect[insect_mask] = bad
    dz_sdp = 1.0 * dzN
    dz_sdp[sdp_mask] = bad
    
    #join masks
    new_mask = np.logical_or(insect_mask, sdp_mask)
    
    #copy reflectivity and apply joined mask
    dz_qc = 1.0 * dzN
    dz_qc[new_mask] = bad
    
    #despeckle then add new field to radar object
    mask_ds = csu_misc.despeckle(dz_qc, ngates=15)
    dz_qc[mask_ds] = bad
    radar = add_field_to_radar_object(dz_qc, radar, field_name='DZ_qc', units='dBZ', 
                                  long_name='Reflectivity (Combo Filtered)',
                                  standard_name='Reflectivity (Combo Filtered)', 
                                  dz_field='reflectivity')
    
    reflect_mask = radar.fields["DZ_qc"]["data"].mask[0:720]
    radar.fields["velocity"]["data"][720:1440].mask = reflect_mask
    #radar.fields["velocity"]["data"][720:1440]
    
    
    #dz_mask = getattr(radar.fields['DZ_qc']['data'], 'mask')
    #dv_mask = getattr(radar.fields['velocity']['data'], 'mask')
    #combined_mask = np.logical_or(dz_mask, dv_mask)
    
    #ve = deepcopy(radar.fields['velocity']['data'])
    #radar.add_field_like('velocity', 'DV_qc', ve, replace_existing=True)
    #setattr(radar.fields['DV_qc']['data'], 'mask', combined_mask)

#     dv_qc = 1.0 * dvN
#     dv_qc[new_mask] = 
#     mask_ds = csu_misc.despeckle(dv_qc, ngates=15)
#     dv_qc[mask_ds] = bad


#    radar.get_field(0,"reflectivity")

#     radar = add_field_to_radar_object(dv_qc, radar, field_name='DV_qc', units='m/s', 
#                                    long_name='Velocity (Combo Filtered)',
#                                    standard_name='Velocity (Combo Filtered)', 
#                                    dz_field='velocity')
    end = time.clock()
    print "Time: ", end - start
    print "Saving ..."
    start = time.clock()
#     cleaned_radar_image(event.Filename,radar, sweep=1, var1='velocity', vmin1=-80, vmax1=80, 
#                cmap1='bwr', units1='m/s',
#                xlim=[-300,300], ylim=[-300,300])
    
    #sqlStatement = "SELECT latitude,longitude FROM locations WHERE episode_id =" + str(event.Episode_ID) + " AND event_id = " + str(event.Event_ID)
    
    #tornadoLoc = pd.read_sql(sqlStatement,con = engine)
        
    
    #box = get_bounding_box(tornadoLoc.latitude[0], tornadoLoc.longitude[0], 120)
    
#     print box.min_lat
#     print box.min_lon
#     print box.max_lat
#     print box.max_lon
#     print "---------"    
#     print tornadoLoc.latitude[0]
#     print tornadoLoc.longitude[0]
    
#     cleaned_radar_map_image(event.Filename, radar, sweep=1, var='velocity', vmin=-80, vmax=80,
#                 cmap='bwr', units='m/s', minLat = box.min_lat, minLong = box.min_lon, maxLat = box.max_lat, maxLong = box.max_lon,
#                 tornadoLat = tornadoLoc.latitude[0], tornadoLong = tornadoLoc.longitude[0])

    vel_flat = radar.fields["velocity"]["data"][720:1440].flatten()
    ref_flat = radar.fields["DZ_qc"]["data"][0:720].flatten()
    #vel_sorted = np.sort(ref_flat)
    #ref_sorted = np.sort(vel_flat)
    
    vel_compressed = vel_flat.compressed()
    ref_compressed = ref_flat.compressed()
    
    # 7 is vel_max
    # 8 is vel_min
    # 9 is ref_max
    # 10 is rev_avg
    # 11 is vel_avg
    # 12 is vel_99
    # 13 is vel_98
    # 14 is ref_99
    # 15 is ref_98
    
    if len(vel_flat.compressed()) != 0:
        data.iloc[i,12] = np.percentile(vel_flat.compressed(),99)
        data.iloc[i,13] = np.percentile(vel_flat.compressed(),98)
            
    if len(ref_flat.compressed())  != 0:
        data.iloc[i,14] = np.percentile(ref_flat.compressed(),99)
        data.iloc[i,15] = np.percentile(ref_flat.compressed(),98)
    
    data.iloc[i,7] = vel_flat.max()
    data.iloc[i,8] = vel_flat.min()
    data.iloc[i,9] = ref_flat.max()
    data.iloc[i,10] = ref_flat.mean()
    data.iloc[i,11] = vel_flat.mean()
    
#     print "Debug: "
#     print i+1
#     print len(vel_maxs)
    
#     if(len(vel_maxs) != i + 1):
#         print "HEY SOMETHING WHENT WRONG"

    end = time.clock()
    print "Time: ", end - start
    #print "Progress: ", round(float(i+1)/(lenOfWeather) * 100,2)
    print "-----------------------------"
    
#     if(i == 0):
#         break

Percent Complete:  0.19
Reading KHTX20150630_154127_V06.gz ...
Cleaning ...
Time:  7.061683
Saving ...
Time:  0.117146
-----------------------------
Percent Complete:  0.38
Reading KHTX20150630_134236_V06.gz ...
Cleaning ...
Time:  5.516354
Saving ...
Time:  0.13385
-----------------------------
Percent Complete:  0.56
Reading KHTX20150626_184539_V06.gz ...
Cleaning ...
Time:  6.898808
Saving ...
Time:  0.08546
-----------------------------
Percent Complete:  0.75
Reading KHTX20150626_194537_V06.gz ...
Cleaning ...
Time:  6.816272
Saving ...
Time:  0.089829
-----------------------------
Percent Complete:  0.94
Reading KBMX20150125_232601_V06.gz ...
Cleaning ...
Time:  4.016038
Saving ...
Time:  0.0856640000001
-----------------------------
Percent Complete:  1.13
Reading KBMX20150125_233152_V06.gz ...
Cleaning ...
Time:  4.138337
Saving ...
Time:  0.0792749999999
-----------------------------
Percent Complete:  1.31
Reading KBMX20150126_001544_V06.gz ...
Cleaning ...
Time:  8.330224
Sa

/Users/alexandertam/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:4144: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


Cleaning ...
Time:  3.201175
Saving ...
Time:  0.0701450000006
-----------------------------
Percent Complete:  74.3
Reading KHTX20150403_230216_V06.gz ...
Cleaning ...
Time:  7.634332
Saving ...
Time:  0.0908980000004
-----------------------------
Percent Complete:  74.48
Reading KHTX20150403_234516_V06.gz ...
Cleaning ...
Time:  8.21857
Saving ...
Time:  0.103822
-----------------------------
Percent Complete:  74.67
Reading KHTX20150403_234516_V06.gz ...
Cleaning ...
Time:  7.84843
Saving ...
Time:  0.0912150000004
-----------------------------
Percent Complete:  74.86
Reading KHTX20150403_234516_V06.gz ...
Cleaning ...
Time:  7.935179
Saving ...
Time:  0.0827060000001
-----------------------------
Percent Complete:  75.05
Reading KHTX20150527_202400_V06.gz ...
Cleaning ...
Time:  6.374542
Saving ...
Time:  0.0947210000004
-----------------------------
Percent Complete:  75.23
Reading KHTX20150404_004915_V06.gz ...
Cleaning ...
Time:  9.036236
Saving ...
Time:  0.0921749999998
-----

In [28]:
data.head()

,Filename,IsTornado,Episode_ID,Event_ID,TornadoTime,VolumeTime,OriginalTime,vel_max,vel_min,ref_max,ref_avg,vel_avg,vel_99,vel_98,rel_99,rel_98
0,KHTX20150630_154127_V06.gz,0,97889,588549,15:45:00,15:41:27,10:45:00,34.5,-64.5,64.5,25.093066,-35.419083,16.5,15.0,51.0,48.5
1,KHTX20150630_134236_V06.gz,0,97889,588550,13:45:00,13:42:36,08:45:00,29.0,-64.5,66.0,26.407601,-47.273160,-1.5,-2.5,53.5,49.0
2,KHTX20150626_184539_V06.gz,0,97885,588529,18:50:00,18:45:39,13:50:00,33.5,-64.5,66.5,27.593727,-48.592673,6.5,5.0,53.5,51.0
3,KHTX20150626_194537_V06.gz,0,97885,588533,19:48:00,19:45:37,14:48:00,28.5,-64.5,64.0,28.013259,-37.455608,10.5,8.5,54.5,52.5
4,KBMX20150125_232601_V06.gz,0,92860,556884,23:30:00,23:26:01,17:30:00,28.5,-64.5,54.5,21.902217,-35.463102,16.0,15.5,43.0,41.0


In [ ]:
#     cleaned_radar_map_image(event.Filename, radar, sweep=1, var='velocity', vmin=-80, vmax=80,
#                 cmap='bwr', units='m/s', minLat = box.min_lat, minLong = box.min_lon, maxLat = box.max_lat, maxLong = box.max_lon,
#                 tornadoLat = tornadoLoc.latitude[0], tornadoLong = tornadoLoc.longitude[0])

In [29]:
y = data.iloc[:,1]
X = data.iloc[:,7:]

In [30]:
# data.to_sql("data",con = engine, if_exists = "replace")

In [32]:
X

,vel_max,vel_min,ref_max,ref_avg,vel_avg,vel_99,vel_98,rel_99,rel_98
0,34.5,-64.5,64.5,25.093066,-35.419083,16.5,15.00,51.0,48.5
1,29.0,-64.5,66.0,26.407601,-47.273160,-1.5,-2.50,53.5,49.0
2,33.5,-64.5,66.5,27.593727,-48.592673,6.5,5.00,53.5,51.0
3,28.5,-64.5,64.0,28.013259,-37.455608,10.5,8.50,54.5,52.5
4,28.5,-64.5,54.5,21.902217,-35.463102,16.0,15.50,43.0,41.0
5,27.5,-64.5,57.5,22.471743,-35.556978,19.0,16.38,45.0,42.0
6,34.5,-64.5,65.0,25.236885,-33.134856,15.5,13.00,51.5,48.5
7,33.5,-64.5,57.0,25.274662,-23.155345,15.5,15.00,44.5,42.0
8,35.0,-64.5,61.5,25.064458,-29.644261,15.0,13.50,50.0,47.5
9,33.5,-64.5,57.0,25.274662,-23.155345,15.5,15.00,44.5,42.0
